# 문서 이미지 분류 베이스라인

## 대회 정보
- **Task**: 문서 이미지 분류 (건강보험증, 여권 등)
- **Train Data**: ~1,500장 | **Test Data**: ~3,000장
- **Metric**: Macro F1 Score | **Framework**: PyTorch

---

## 🎯 추천 모델 (실험 순서)

### 1단계: Baseline ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b0'
```
**파라미터**: 5M | **속도**: ~1분/epoch | **용도**: 가장 가볍고 빠른 시작점

### 2단계: 성능 향상 ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b1'  # 추천 1순위
# 또는
CFG.model_name = 'efficientnet_b2'  # 추천 2순위
```
**B1**: 7M, ~1.5분/epoch, B0 대비 +2~3% 향상  
**B2**: 9M, ~2분/epoch, B0 대비 +3~5% 향상

### 3단계: 최신 아키텍처 ⭐⭐⭐⭐⭐
```python
CFG.model_name = 'convnext_tiny'
```
**파라미터**: 28M | **속도**: ~2.5분/epoch | **특징**: 최신(2022), B0 대비 +5~7% 향상

### 4단계: 성능 극대화 ⭐⭐⭐⭐
```python
CFG.model_name = 'efficientnet_b3'  # 1순위
# 또는
CFG.model_name = 'convnext_small'   # 2순위 (최고 성능)
```
**B3**: 12M, ~2.5분/epoch, B0 대비 +5~8% 향상  
**ConvNeXt-Small**: 50M, ~4분/epoch, B0 대비 +7~10% 향상

### 5단계: Transformer (선택) ⭐⭐
```python
CFG.model_name = 'vit_base_patch16_224'
# 또는
CFG.model_name = 'swin_base_patch4_window7_224'
```
**ViT**: 86M, ~5분/epoch | **Swin**: 88M, ~5분/epoch  
**주의**: 1,500장에선 과적합 위험 높음, 강한 증강 필요, **비추천**

---

## ⚠️ 비추천 모델
- `resnet50` / `resnet101` - EfficientNet보다 비효율적
- `mobilenetv3_large_100` - 속도 빠르지만 성능 낮음
- `vit` / `swin` - 데이터 부족 시 과적합 (10,000장 이상일 때 추천)

---

## 🚀 실험 시나리오

### 시나리오 1: 빠른 실험 (2시간)
1. B0 + medium 증강 → 30분
2. B0 + light 증강 → 30분
3. B0 + heavy 증강 → 30분
4. B1 + 최고 증강 → 40분

### 시나리오 2: 균형 실험 (4시간)
B0(3가지 증강) → B1 → B2 → ConvNeXt-Tiny → 최고 모델 재학습

### 시나리오 3: 최고 성능 (하루)
B0 최적화 → B1/B2 → ConvNeXt-Tiny → B3 → ConvNeXt-Small → 앙상블

---

## 🔧 트러블슈팅

**GPU 메모리 부족**: `batch_size = 16` 또는 `img_size = 192`  
**학습 느림**: `epochs = 20` (EfficientNet-B0/B1은 20 epoch 충분)  
**성능 plateau**: 모델 크기보다 증강/하이퍼파라미터 튜닝 먼저!


## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필요한 라이브러리 설치
!pip install timm wandb -q

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import timm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix

import wandb

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 2. 시드 고정 (재현성)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

## 3. 구글 드라이브 마운트 (선택사항)

In [ ]:
# 구글 드라이브 마운트 (데이터나 모델을 드라이브에 저장하려면 실행)
# 실행하면 인증 링크가 나타나고, 권한 승인 후 코드를 붙여넣으면 됩니다.

from google.colab import drive
drive.mount('/content/drive')

print("구글 드라이브가 /content/drive 에 마운트되었습니다.")
print("데이터 경로 예시: /content/drive/MyDrive/your_data_folder/")

## 3. WandB 초기화

In [ ]:
# WandB 로그인 (처음 실행시 API 키 입력 필요)
# https://wandb.ai/authorize 에서 API 키 발급
wandb.login()

# 프로젝트명은 실제 대회명으로 변경하세요
WANDB_PROJECT = "document-classification"
WANDB_ENTITY = None  # 팀 계정 사용시 팀명 입력

## 4. 하이퍼파라미터 설정

In [ ]:
# 모델별 권장 이미지 사이즈 (문서 이미지 최적화)
# 문서 이미지는 텍스트와 세밀한 디테일이 중요하므로 일반 이미지보다 큰 사이즈 사용
MODEL_IMG_SIZES = {
    'efficientnet_b0': 384,   # 기본 224 → 384로 증가
    'efficientnet_b1': 416,   # 기본 240 → 416으로 증가
    'efficientnet_b2': 448,   # 기본 260 → 448로 증가
    'efficientnet_b3': 512,   # 기본 300 → 512로 증가
    'efficientnet_b4': 512,   # 기본 380 → 512 유지
    'convnext_tiny': 384,     # 기본 224 → 384로 증가
    'convnext_small': 384,    # 기본 224 → 384로 증가
    'vit_base_patch16_224': 384,  # 기본 224 → 384로 증가
    'swin_base_patch4_window7_224': 384,  # 기본 224 → 384로 증가
}

class CFG:
    # 데이터 경로
    train_dir = './data/train'  # 학습 이미지 폴더
    test_dir = './data/test'    # 테스트 이미지 폴더
    
    # 모델 설정
    model_name = 'efficientnet_b0'  # timm 모델명
    num_classes = 10  # 실제 클래스 개수로 변경 필요
    img_size = MODEL_IMG_SIZES.get(model_name, 384)  # 모델별 권장 사이즈 자동 적용 (문서 이미지용)
    
    # 학습 설정
    epochs = 30
    batch_size = 32
    learning_rate = 1e-4
    weight_decay = 1e-5
    
    # Early Stopping 설정
    early_stopping_patience = 3  # 3 epoch 동안 개선 없으면 중단
    early_stopping_min_delta = 0.0001  # F1 차이 0.01% 미만은 개선 아님
    
    # 데이터 분할
    val_ratio = 0.2
    
    # 모델 저장 경로
    save_to_drive = True  # 구글 드라이브에 저장 여부
    drive_model_dir = '/content/drive/MyDrive/document_classification/models'  # 드라이브 저장 경로
    local_model_path = 'best_model.pth'  # 로컬 저장 경로
    
    # WandB 설정
    use_wandb = True
    wandb_project = WANDB_PROJECT
    wandb_entity = WANDB_ENTITY
    experiment_name = None  # None이면 자동으로 번호 부여
    
    # 실험명 접두사 설정
    # 옵션 1: 모델명 자동 사용 (기본, None으로 두면 자동)
    experiment_prefix = None  # None이면 model_name 사용
    
    # 옵션 2: 커스텀 prefix 사용 (필요시 아래 주석 해제)
    # experiment_prefix = 'baseline'  # baseline_001, baseline_002 ...
    # experiment_prefix = 'augment'   # augment_001, augment_002 ...
    
    # 기타
    num_workers = 2
    seed = 42

## 5. 데이터셋 클래스

In [ ]:
class DocumentDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

## 6. 데이터 증강 (Augmentation)

In [ ]:
# 학습용 Transform
train_transform = transforms.Compose([
    transforms.Resize((CFG.img_size, CFG.img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

# 검증/테스트용 Transform
val_transform = transforms.Compose([
    transforms.Resize((CFG.img_size, CFG.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

## 7. 데이터 로드 및 전처리

**주의**: 데이터 폴더 구조에 맞게 수정이 필요합니다.

예상 구조:
```
data/
├── train/
│   ├── class1/
│   ├── class2/
│   └── ...
└── test/
    ├── image1.jpg
    ├── image2.jpg
    └── ...
```

또는 CSV 파일이 있다면:
```python
train_df = pd.read_csv('train.csv')
# train_df: ['image_path', 'label'] 컬럼 포함
```

In [ ]:
# 방법 1: 폴더 구조로부터 데이터 로드
def load_data_from_folders(data_dir):
    image_paths = []
    labels = []
    
    class_names = sorted(os.listdir(data_dir))
    class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
            
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                image_paths.append(img_path)
                labels.append(class_to_idx[class_name])
    
    return image_paths, labels, class_to_idx

# 학습 데이터 로드
if os.path.exists(CFG.train_dir):
    train_paths, train_labels, class_to_idx = load_data_from_folders(CFG.train_dir)
    print(f"Total training images: {len(train_paths)}")
    print(f"Number of classes: {len(class_to_idx)}")
    print(f"Classes: {class_to_idx}")
    
    # CFG.num_classes 업데이트
    CFG.num_classes = len(class_to_idx)
else:
    print(f"Warning: {CFG.train_dir} does not exist!")
    print("Please upload your data or modify the path.")

In [ ]:
# 방법 2: CSV 파일로부터 데이터 로드 (필요시 사용)
# train_df = pd.read_csv('train.csv')
# train_paths = train_df['image_path'].tolist()
# train_labels = train_df['label'].tolist()

In [ ]:
# Train/Validation 분할
if 'train_paths' in locals():
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_paths, train_labels, 
        test_size=CFG.val_ratio, 
        random_state=CFG.seed,
        stratify=train_labels
    )
    
    print(f"Train size: {len(train_paths)}")
    print(f"Validation size: {len(val_paths)}")

In [ ]:
# 데이터셋 및 데이터로더 생성
if 'train_paths' in locals():
    train_dataset = DocumentDataset(train_paths, train_labels, train_transform)
    val_dataset = DocumentDataset(val_paths, val_labels, val_transform)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CFG.batch_size, 
        shuffle=True, 
        num_workers=CFG.num_workers
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=CFG.batch_size, 
        shuffle=False, 
        num_workers=CFG.num_workers
    )

## 8. WandB Run 초기화 (학습 시작 전)

In [ ]:
def get_next_experiment_number(project_name, prefix, entity=None):
    """WandB에서 기존 실험들을 확인하고 다음 번호를 반환"""
    try:
        api = wandb.Api()
        # 프로젝트의 모든 run 가져오기
        if entity:
            runs = api.runs(f"{entity}/{project_name}")
        else:
            runs = api.runs(project_name)
        
        # prefix로 시작하는 run들의 번호 추출
        numbers = []
        for run in runs:
            if run.name.startswith(prefix):
                try:
                    # 'prefix_123' 형태에서 123 추출
                    num = int(run.name.split('_')[-1])
                    numbers.append(num)
                except:
                    continue
        
        # 가장 큰 번호 + 1 반환
        next_num = max(numbers) + 1 if numbers else 1
        return next_num
    except:
        # API 접근 실패시 001부터 시작
        return 1

# WandB Run 초기화
if CFG.use_wandb:
    # experiment_prefix가 None이면 모델명 사용 (자동)
    if CFG.experiment_prefix is None:
        actual_prefix = CFG.model_name
    else:
        actual_prefix = CFG.experiment_prefix
    
    # 실험명 자동 생성
    if CFG.experiment_name is None:
        exp_num = get_next_experiment_number(
            CFG.wandb_project, 
            actual_prefix,
            CFG.wandb_entity
        )
        CFG.experiment_name = f"{actual_prefix}_{exp_num:03d}"
    
    run = wandb.init(
        project=CFG.wandb_project,
        entity=CFG.wandb_entity,
        name=CFG.experiment_name,
        config={
            "model_name": CFG.model_name,
            "num_classes": CFG.num_classes,
            "img_size": CFG.img_size,
            "epochs": CFG.epochs,
            "batch_size": CFG.batch_size,
            "learning_rate": CFG.learning_rate,
            "weight_decay": CFG.weight_decay,
            "optimizer": "AdamW",
            "scheduler": "CosineAnnealingLR",
            "val_ratio": CFG.val_ratio,
            "seed": CFG.seed,
        }
    )
    print(f"\n{'='*60}")
    print(f"WandB Run initialized: {run.name}")
    print(f"WandB URL: {run.url}")
    print(f"{'='*60}\n")
else:
    print("WandB is disabled")

## 9. 모델 정의

In [ ]:
class DocumentClassifier(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True):
        super(DocumentClassifier, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        
        # 모델의 classifier 부분 수정
        if 'efficientnet' in model_name:
            in_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(in_features, num_classes)
        elif 'resnet' in model_name:
            in_features = self.model.fc.in_features
            self.model.fc = nn.Linear(in_features, num_classes)
        elif 'vit' in model_name:
            in_features = self.model.head.in_features
            self.model.head = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# 모델 생성
model = DocumentClassifier(
    model_name=CFG.model_name, 
    num_classes=CFG.num_classes, 
    pretrained=True
).to(device)

print(f"Model: {CFG.model_name}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

# WandB에 모델 아키텍처 로깅
if CFG.use_wandb:
    wandb.watch(model, log='all', log_freq=100)

## 10. 손실 함수 및 옵티마이저

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=CFG.learning_rate, weight_decay=CFG.weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs, eta_min=1e-6)

## 11. 학습 및 검증 함수

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 배치별 메트릭 계산
        batch_loss = running_loss / (batch_idx + 1)
        batch_acc = 100. * correct / total
        
        pbar.set_postfix({
            'loss': batch_loss,
            'acc': batch_acc
        })
        
        # WandB 로깅 (매 배치마다)
        if CFG.use_wandb:
            wandb.log({
                'train/batch_loss': loss.item(),
                'train/batch_acc': 100. * predicted.eq(labels).sum().item() / labels.size(0),
                'train/step': epoch * len(train_loader) + batch_idx
            })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    
    # Macro F1 Score 계산
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    
    # 클래스별 F1 Score 계산
    per_class_f1 = f1_score(all_labels, all_preds, average=None)
    
    # Confusion Matrix 계산
    cm = confusion_matrix(all_labels, all_preds)
    
    return epoch_loss, macro_f1, per_class_f1, cm, all_preds, all_labels

## 12. 학습 실행

In [ ]:
best_f1 = 0.0
patience_counter = 0  # Early Stopping 카운터
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_f1': []
}

# 구글 드라이브 저장 경로 생성
if CFG.save_to_drive:
    os.makedirs(CFG.drive_model_dir, exist_ok=True)
    print(f"모델 저장 경로: {CFG.drive_model_dir}")

print(f"\n{'='*60}")
print(f"Early Stopping: Patience={CFG.early_stopping_patience}, Min Delta={CFG.early_stopping_min_delta}")
print(f"{'='*60}\n")

for epoch in range(CFG.epochs):
    print(f"\nEpoch {epoch+1}/{CFG.epochs}")
    print("-" * 50)
    
    # 학습
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # 검증
    val_loss, val_f1, per_class_f1, cm, val_preds, val_labels = validate(model, val_loader, criterion, device)
    
    # 스케줄러 업데이트
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # 결과 저장
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Macro F1: {val_f1:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")
    
    # WandB 로깅 (에폭별)
    if CFG.use_wandb:
        # 기본 메트릭
        log_dict = {
            'epoch': epoch + 1,
            'train/epoch_loss': train_loss,
            'train/epoch_acc': train_acc,
            'val/loss': val_loss,
            'val/macro_f1': val_f1,
            'learning_rate': current_lr,
            'early_stopping/patience_counter': patience_counter,
        }
        
        # 클래스별 F1 Score
        if 'class_to_idx' in locals():
            idx_to_class = {v: k for k, v in class_to_idx.items()}
            for idx, f1 in enumerate(per_class_f1):
                class_name = idx_to_class.get(idx, f'class_{idx}')
                log_dict[f'val/f1_{class_name}'] = f1
        
        # Confusion Matrix (5 에폭마다)
        if (epoch + 1) % 5 == 0:
            log_dict['val/confusion_matrix'] = wandb.plot.confusion_matrix(
                probs=None,
                y_true=val_labels,
                preds=val_preds,
                class_names=[idx_to_class.get(i, f'class_{i}') for i in range(CFG.num_classes)] if 'idx_to_class' in locals() else None
            )
        
        wandb.log(log_dict)
    
    # Early Stopping 체크 및 베스트 모델 저장
    if val_f1 > best_f1 + CFG.early_stopping_min_delta:
        # 성능 개선됨
        best_f1 = val_f1
        patience_counter = 0  # 카운터 리셋
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_f1': best_f1,
        }
        
        # 로컬에 저장
        torch.save(checkpoint, CFG.local_model_path)
        print(f"✓ Best model saved locally! (F1: {best_f1:.4f})")
        
        # 구글 드라이브에 저장
        if CFG.save_to_drive:
            # 실험명을 파일명에 포함
            if CFG.use_wandb and CFG.experiment_name:
                drive_model_path = f"{CFG.drive_model_dir}/{CFG.experiment_name}_f1_{best_f1:.4f}.pth"
            else:
                drive_model_path = f"{CFG.drive_model_dir}/best_model_f1_{best_f1:.4f}.pth"
            
            torch.save(checkpoint, drive_model_path)
            print(f"✓ Best model saved to drive: {drive_model_path}")
        
        # WandB에 베스트 모델 저장
        if CFG.use_wandb:
            artifact = wandb.Artifact(
                name=f'model-{run.id}',
                type='model',
                description=f'Best model with F1: {best_f1:.4f}',
                metadata={
                    'epoch': epoch + 1,
                    'val_f1': val_f1,
                    'val_loss': val_loss,
                }
            )
            artifact.add_file(CFG.local_model_path)
            wandb.log_artifact(artifact)
    else:
        # 성능 개선 없음
        patience_counter += 1
        print(f"⚠ No improvement. Patience: {patience_counter}/{CFG.early_stopping_patience}")
        
        # Early Stopping 체크
        if patience_counter >= CFG.early_stopping_patience:
            print(f"\n{'='*60}")
            print(f"Early Stopping triggered at epoch {epoch+1}")
            print(f"Best Validation Macro F1: {best_f1:.4f}")
            print(f"{'='*60}")
            break

print(f"\n{'='*60}")
print(f"Training completed!")
print(f"Best Validation Macro F1: {best_f1:.4f}")
print(f"Total epochs: {epoch+1}")
if patience_counter >= CFG.early_stopping_patience:
    print(f"Stopped early due to no improvement for {CFG.early_stopping_patience} epochs")
print(f"{'='*50}")

# 최종 모델 경로 출력
print(f"\n모델 저장 위치:")
print(f"  - 로컬: {CFG.local_model_path}")
if CFG.save_to_drive:
    print(f"  - 드라이브: {CFG.drive_model_dir}/")

# WandB Run 종료
if CFG.use_wandb:
    wandb.finish()

## 13. 학습 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss 그래프
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# F1 Score 그래프
axes[1].plot(history['val_f1'], label='Val Macro F1', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Macro F1 Score')
axes[1].set_title('Validation Macro F1 Score')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 14. 테스트 데이터 추론

In [ ]:
# 베스트 모델 로드
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Best model loaded (F1: {checkpoint['best_f1']:.4f})")

In [ ]:
# 테스트 데이터 로드
def load_test_data(test_dir):
    test_paths = []
    for img_name in sorted(os.listdir(test_dir)):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            test_paths.append(os.path.join(test_dir, img_name))
    return test_paths

if os.path.exists(CFG.test_dir):
    test_paths = load_test_data(CFG.test_dir)
    print(f"Total test images: {len(test_paths)}")
else:
    print(f"Warning: {CFG.test_dir} does not exist!")
    test_paths = []

In [ ]:
# 테스트 데이터셋 및 로더 생성
if test_paths:
    test_dataset = DocumentDataset(test_paths, labels=None, transform=val_transform)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=CFG.batch_size, 
        shuffle=False, 
        num_workers=CFG.num_workers
    )

In [ ]:
# 추론 함수
def predict(model, test_loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for images in tqdm(test_loader, desc='Predicting'):
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())
    
    return predictions

# 추론 실행
if test_paths:
    predictions = predict(model, test_loader, device)
    print(f"Prediction completed: {len(predictions)} samples")

## 15. 제출 파일 생성

In [ ]:
# 제출 파일 생성 (형식은 대회 규정에 맞게 수정)
if test_paths and predictions:
    # 클래스 인덱스를 클래스 이름으로 변환
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    submission = pd.DataFrame({
        'image': [os.path.basename(path) for path in test_paths],
        'label': [idx_to_class[pred] for pred in predictions]
    })
    
    # 또는 숫자 레이블로 제출하는 경우:
    # submission = pd.DataFrame({
    #     'image': [os.path.basename(path) for path in test_paths],
    #     'label': predictions
    # })
    
    submission.to_csv('submission.csv', index=False)
    print("\nSubmission file saved: submission.csv")
    print(submission.head(10))

## 16. 추가 개선 아이디어

### 모델 개선
1. **다양한 모델 시도**
   - `efficientnet_b3`, `efficientnet_b4` (더 큰 모델)
   - `convnext_tiny`, `convnext_small`
   - `vit_base_patch16_224` (Vision Transformer)
   - `swin_base_patch4_window7_224`

2. **앙상블**
   - 여러 모델의 예측을 결합 (Voting, Averaging)
   - K-Fold Cross Validation

3. **데이터 증강 강화**
   - AutoAugment, RandAugment
   - MixUp, CutMix
   - Test Time Augmentation (TTA)

4. **학습 기법**
   - Label Smoothing
   - Focal Loss (클래스 불균형 시)
   - Stochastic Weight Averaging (SWA)
   - Gradient Accumulation (배치 크기 확장)

5. **이미지 전처리**
   - 문서 정렬 (Document alignment)
   - 해상도 조정
   - 노이즈 제거

### WandB 활용 팁
1. **Sweep을 이용한 하이퍼파라미터 튜닝**
   - 자동으로 최적의 하이퍼파라미터 찾기
   - Bayesian Optimization, Random Search 등 지원

2. **실험 비교**
   - 여러 실험을 한눈에 비교
   - Parallel coordinates plot으로 관계 분석

3. **팀 협업**
   - 팀원과 실험 결과 공유
   - 코멘트 및 노트 기능

## 코랩 환경 팁

### 1. GPU 런타임 사용
상단 메뉴: `런타임` → `런타임 유형 변경` → `하드웨어 가속기: GPU`

### 2. 구글 드라이브 마운트
```python
from google.colab import drive
drive.mount('/content/drive')
```

### 3. 데이터 업로드
- 좌측 파일 탭에서 업로드
- 또는 구글 드라이브에서 읽기
- Kaggle API 사용 (Kaggle 대회인 경우)

### 4. 모델 및 결과 저장
```python
# 구글 드라이브에 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/models/best_model.pth')
```

### 5. WandB 실험명 자동 번호 부여 사용법 (중요!)

#### 📌 기본 개념
이 노트북은 **실험명을 자동으로 번호를 매겨서 관리**하는 기능이 내장되어 있습니다.
**기본 설정: 모델명만 바꾸면 자동으로 실험명도 변경됩니다!** ⭐

#### ✨ 사용법 (매우 간단!)

**모델만 바꾸면 끝!**
```python
# 섹션 4의 CFG 클래스에서
CFG.model_name = 'efficientnet_b0'  # ← 이것만 바꾸면 됩니다!
```
→ 자동으로 `efficientnet_b0_001`, `efficientnet_b0_002` ...

**다른 모델로 실험하고 싶다면?**
```python
CFG.model_name = 'resnet50'  # ← 모델명만 변경
```
→ 자동으로 `resnet50_001`, `resnet50_002` ...

그게 다입니다! 나머지는 자동으로 처리됩니다. 😊

#### 🔧 상세 설정 (선택사항)

**방법 1: 모델명 자동 사용 (기본값, 권장) ⭐**
```python
# 섹션 4의 CFG 클래스에서
CFG.model_name = 'efficientnet_b0'
CFG.experiment_prefix = None  # None으로 유지 (기본값)
CFG.experiment_name = None    # None으로 유지 (기본값)
```
→ 결과: `efficientnet_b0_001`, `efficientnet_b0_002` ...

**방법 2: 커스텀 prefix 사용**
```python
# 모델명 대신 다른 이름을 사용하고 싶을 때
CFG.experiment_prefix = 'baseline'
```
→ 결과: `baseline_001`, `baseline_002` ...

**방법 3: 완전히 수동으로 실험명 지정**
```python
# 특정 실험에 의미있는 이름을 붙이고 싶을 때
CFG.experiment_name = 'final_submission_v1'
```
→ 결과: `final_submission_v1` (지정한 이름 그대로)

#### 💡 실험 시나리오별 활용 예시

**시나리오 1: 다양한 모델 비교 (자동 관리)**
```python
# EfficientNet B0 실험
CFG.model_name = 'efficientnet_b0'  # ← 이것만 바꾸면 됨!
# → efficientnet_b0_001, efficientnet_b0_002 ...

# EfficientNet B3 실험
CFG.model_name = 'efficientnet_b3'  # ← 이것만 바꾸면 됨!
# → efficientnet_b3_001, efficientnet_b3_002 ...

# ResNet50 실험
CFG.model_name = 'resnet50'  # ← 이것만 바꾸면 됨!
# → resnet50_001, resnet50_002 ...
```

**시나리오 2: 전략별 실험 (커스텀 prefix)**
```python
# 기본 베이스라인
CFG.experiment_prefix = 'baseline'
# → baseline_001, baseline_002 ...

# 강한 데이터 증강 실험
CFG.experiment_prefix = 'strong_augment'
# → strong_augment_001, strong_augment_002 ...

# 앙상블 실험
CFG.experiment_prefix = 'ensemble'
# → ensemble_001, ensemble_002 ...
```

**시나리오 3: 같은 모델로 다양한 하이퍼파라미터 실험**
```python
# EfficientNet B0로 여러 학습률 실험
CFG.model_name = 'efficientnet_b0'
CFG.experiment_prefix = 'effb0_lr_tuning'  # 커스텀 prefix 사용

CFG.learning_rate = 1e-4
# → effb0_lr_tuning_001

CFG.learning_rate = 1e-5
# → effb0_lr_tuning_002

CFG.learning_rate = 5e-5
# → effb0_lr_tuning_003
```

#### 📊 실행 예시

**첫 번째 실험 (EfficientNet B0):**
```python
CFG.model_name = 'efficientnet_b0'
```
출력:
```
============================================================
WandB Run initialized: efficientnet_b0_001
WandB URL: https://wandb.ai/username/document-classification/runs/xxx
============================================================
```

**같은 모델로 두 번째 실험:**
```
============================================================
WandB Run initialized: efficientnet_b0_002
WandB URL: https://wandb.ai/username/document-classification/runs/yyy
============================================================
```

**모델 변경 후 실험:**
```python
CFG.model_name = 'efficientnet_b3'
```
출력:
```
============================================================
WandB Run initialized: efficientnet_b3_001
WandB URL: https://wandb.ai/username/document-classification/runs/zzz
============================================================
```

#### ⚙️ 작동 원리
1. `CFG.experiment_prefix = None`이면 → `CFG.model_name`을 자동으로 사용
2. `CFG.experiment_name = None`이면 → 자동 번호 부여 모드 활성화
3. WandB API를 통해 프로젝트의 기존 실험들을 확인
4. prefix로 시작하는 실험 중 가장 큰 번호 찾기
5. 가장 큰 번호 + 1로 새 실험명 생성
6. 해당 prefix의 첫 실험이면 001부터 시작

#### 🎯 실전 팁
- **초간단 사용**: `CFG.model_name`만 바꾸면 모든 게 자동입니다! ⭐
- **모델 비교**: 각 모델별로 자동으로 그룹화되어 비교가 쉽습니다
- **전략 비교**: 같은 전략의 실험들을 묶고 싶으면 `experiment_prefix`를 직접 지정하세요
- **WandB 필터링**: WandB 대시보드에서 prefix별로 필터링하면 비교가 쉽습니다
- **재실행**: 코랩 런타임이 끊겨도 번호는 계속 이어집니다 (WandB 서버에 저장되므로)

#### 📋 빠른 참조 테이블

| 상황 | 설정 방법 | 결과 예시 |
|------|-----------|-----------|
| **기본 사용 (모델명 자동)** | `CFG.model_name = 'efficientnet_b0'`만 설정 | `efficientnet_b0_001` |
| **다른 모델로 변경** | `CFG.model_name = 'resnet50'`으로 변경 | `resnet50_001` |
| 전략별 그룹화 | `CFG.experiment_prefix = 'baseline'` | `baseline_001` |
| 수동 이름 지정 | `CFG.experiment_name = 'my_best_model'` | `my_best_model` |
| 하이퍼파라미터 튜닝 | `CFG.experiment_prefix = 'lr_tuning'` | `lr_tuning_001` |

#### 🚀 실전 워크플로우 예시

**1일차: EfficientNet B0 실험**
```python
CFG.model_name = 'efficientnet_b0'
# 학습 실행 → efficientnet_b0_001
```

**2일차: 하이퍼파라미터 조정**
```python
CFG.model_name = 'efficientnet_b0'  # 그대로
CFG.learning_rate = 5e-5  # 변경
# 학습 실행 → efficientnet_b0_002
```

**3일차: 더 큰 모델로 실험**
```python
CFG.model_name = 'efficientnet_b3'  # 모델만 변경!
# 학습 실행 → efficientnet_b3_001
```

**4일차: ResNet도 시도**
```python
CFG.model_name = 'resnet50'  # 모델만 변경!
# 학습 실행 → resnet50_001
```

**5일차: 최고 성능 모델 재학습**
```python
CFG.model_name = 'efficientnet_b3'
CFG.experiment_name = 'final_submission'  # 수동 지정
# 학습 실행 → final_submission
```

### 6. WandB 일반 사용법
1. **첫 실행시**: `wandb.login()` 실행 후 https://wandb.ai/authorize 에서 API 키 복사/붙여넣기
2. **실험 추적**: 학습 중 WandB 대시보드에서 실시간 모니터링
3. **결과 확인**: 학습 완료 후 WandB URL에서 모든 메트릭과 차트 확인
4. **실험 비교**: 여러 실험을 선택해서 성능 비교 가능